In [1]:
%matplotlib inline 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from pathlib import Path
import copy

import os
import torch

import typhon_model


In [24]:
root_path = Path('results')
bootstrap_exp_name = '20230513_bootstrap_7'

model_path = (root_path / bootstrap_exp_name / 'models' / 'bootstrap_model').with_suffix('.pth')
print(model_path)

# Load model
model_dict = torch.load(model_path, map_location=torch.device('cpu'))
model = typhon_model.TyphonModel.from_state_dict(model_dict)

results/20230513_bootstrap_7/models/bootstrap_model.pth


In [25]:
new_model = copy.deepcopy(model)

### Remove BUS_SELECTED module

### Change name to BUSI --> UDIAT

In [26]:
new_model.dms.update({'UDIAT': new_model.dms['BUSI']})

In [27]:
new_model.dms.pop('BUS_SELECTED')
new_model.dms.pop('BUSI')
print()

### Update names

In [28]:
new_model.dsets_names = [k for k in new_model.dms.keys()]

### Adapt dropout infos

In [29]:
new_model.dropouts_dm['UDIAT'] = new_model.dropouts_dm['BUSI']

In [30]:
new_model.dropouts_dm.pop('BUS_SELECTED')
new_model.dropouts_dm.pop('BUSI')
print()

### Check

In [31]:
new_model.dms.keys()

odict_keys(['LINK_BRATS_LGG_flair', 'BRAIN', 'UDIAT'])

In [32]:
new_model.dsets_names

['LINK_BRATS_LGG_flair', 'BRAIN', 'UDIAT']

In [33]:
new_model.dropouts_dm

{'LINK_BRATS_LGG_flair': 0.1, 'BRAIN': 0.1, 'UDIAT': 0.1}

## Save new model

In [34]:
OUTPUT_FOLDER = root_path / 'bootstrap_models_adapted' / bootstrap_exp_name 
assert not os.path.exists(OUTPUT_FOLDER)
os.mkdir(OUTPUT_FOLDER)
torch.save(new_model.to_state_dict(), OUTPUT_FOLDER / 'bootstrap_model.pth')